In [192]:

import csv
from csv import DictReader

data_dblp = []
with open('Desktop/data_dummy.csv', encoding="utf8", errors='ignore') as read_obj:
    dict_reader = DictReader(read_obj)
    data_dblp = list(dict_reader)

data_dblp = data_dblp[:80]
print(len(data_dblp))


80


In [182]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model

In [183]:
pairsData = []

for data in data_dblp:
    pairsData.append(tuple((data['paperAbstract1'], data['paperAbstract2'], data['label'])))

print(len(pairsData))
print(pairsData[0][1])

80
The emergence of advanced technologies such as InfiniBand and Non-Volatile Memory together with decreasing in DRAM prices has enabled us to build ultra-low latency in-memory stores for backing real-time and large-scale services. Many studies have taken those opportunities to accelerate lookup operations of key-value stores. Meanwhile, there is a lack of researches that utilize them for improving range query performance. This paper reports a detailed performance analysis of range query execution of in-memory stores. Our findings show that in modern architecture, copying/serialization data for transmission over network is the major overhead of range query processing. To solve this issue, several methods could be considered, such as parallelism, caching, and utilizing RDMA Read. We have conducted some experiments to evaluate their potentials and surprisingly, the outcomes revealed that none of those performed well in every scenario. Optimizing performance gain when employing those tech

In [184]:
source_index = {data[0]: idx for idx, data in enumerate(pairsData)}
target_index = {data[1]: idx for idx, data in enumerate(pairsData)}


In [185]:
#print(source_index)
#print(target_index)

In [186]:
finalData = []

for data in data_dblp:
    finalData.append(tuple((source_index[data['paperAbstract1']], target_index[data['paperAbstract2']], data['label'])))
    
print(len(finalData))

80


In [187]:
embedding_size = 5
def model():
    
    source = Input(name = 'source', shape = [1])
    target = Input(name = 'target', shape = [1])
    
    source_embedding = Embedding(name = 'source_embedding',input_dim = len(source_index),
                               output_dim = embedding_size)(source)
    
    target_embedding = Embedding(name = 'target_embedding',input_dim = len(target_index),
                               output_dim = embedding_size)(target)
    
    final_layer = Reshape([1])(Dot(name = 'dot_product', normalize = True, axes = 2)([source_embedding, target_embedding]))
    
    final_layer = Dense(1, activation = 'sigmoid')(final_layer)
    model = Model(inputs = [source, target], outputs = final_layer)
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

model = model()
model.summary()

Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
source (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
target (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
source_embedding (Embedding)    (None, 1, 5)         400         source[0][0]                     
__________________________________________________________________________________________________
target_embedding (Embedding)    (None, 1, 5)         350         target[0][0]                     
______________________________________________________________________________________

In [188]:
import numpy as np
import random
random.seed(100)

batch_size = 8
print(len(finalData))
def batchifier(finalData):
    """Generate batches of samples for training"""
    batch = np.zeros((batch_size, 3))
    
    while True:
        for idx, (source_id, target_id, label) in enumerate(random.sample(finalData, batch_size)):
            batch[idx, :] = (source_id, target_id, label)

        np.random.shuffle(batch)
        #print(len(batch))
        yield {'source': batch[:, 0], 'target': batch[:, 1]}, batch[:, 2]

80


In [189]:
gen = batchifier(finalData)

In [ ]:
model = model.fit_generator(gen, epochs = 2, steps_per_epoch = 2,verbose = 2)